In [22]:
import os
from openai import OpenAI
from pathlib import Path

from slugify import slugify

SYSTEM_PROMPTS = Path("src")/"llm"/"system_prompts"
USER_PROMPTS = Path("src")/"llm"/"user_prompts"
OUTPUT_DIR = Path("outputs")

from src.utils.config import settings
from src.utils.logging import configure_logging, get_logger
configure_logging(
    log_level="INFO",
    json_format=False,
)
LOGGER = get_logger("notebook")


CLIENT = OpenAI(
    base_url = f"http://{settings.openai_api.host}:{settings.openai_api.port}/v1",
    api_key = f"{settings.openai_api.api_key}"
)




In [27]:
models = [
    # "hf.co/lmstudio-community/gemma-3-4B-it-qat-GGUF",
    # "hf.co/lmstudio-community/gemma-3-12b-it-GGUF:Q6_K"
    # "hf.co/lmstudio-community/gemma-3-12b-it-GGUF",
    "hf.co/lmstudio-community/gemma-3-12B-it-qat-GGUF"
]
temperatures = [0.0, 1.3, 2.0]

system_prompt = ""
with open(SYSTEM_PROMPTS/"general.md") as file:
    system_prompt = file.read()

user_prompt = "tell me about stocashtic indicators"
slug = slugify(user_prompt, max_length=40)

for model in models:
    model_name = Path(model).name
    for i in range(1, 2):
        for temperature in temperatures:
            LOGGER.info(f"requesting {model_name} at {temperature}")
            response = CLIENT.chat.completions.create(
                messages=[
                    {
                        'role': 'system',
                        'content': system_prompt
                    },
                    {
                        "role": 'user',
                        "content": user_prompt
                    }
                ],
                model=model,
                temperature=temperature
            )
            if len(response.choices) > 1:
                LOGGER.warning(f"generated {len(response.choices)} choices")

            output = response.choices[0].message.content

            output_path = OUTPUT_DIR/slug
            output_path.mkdir(exist_ok=True)
            filename = f"{model_name} at {temperature} ({i}).md"
            with open(output_path/filename, "w") as file:
                file.write(output)
            LOGGER.info(f"wrote {len(output)} to {output_path}/{filename}")






2025-04-22T06:45:04.784519Z [info     ] requesting gemma-3-12B-it-qat-GGUF at 0.0
HTTP Request: POST http://10.0.0.4:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-22T06:45:40.381323Z [info     ] wrote 5611 to outputs/tell-me-about-stocashtic-indicators/gemma-3-12B-it-qat-GGUF at 0.0 (1).md
2025-04-22T06:45:40.382126Z [info     ] requesting gemma-3-12B-it-qat-GGUF at 1.3
HTTP Request: POST http://10.0.0.4:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-22T06:46:19.137906Z [info     ] wrote 6199 to outputs/tell-me-about-stocashtic-indicators/gemma-3-12B-it-qat-GGUF at 1.3 (1).md
2025-04-22T06:46:19.138753Z [info     ] requesting gemma-3-12B-it-qat-GGUF at 2.0
HTTP Request: POST http://10.0.0.4:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-22T06:47:00.938613Z [info     ] wrote 7232 to outputs/tell-me-about-stocashtic-indicators/gemma-3-12B-it-qat-GGUF at 2.0 (1).md


In [33]:
from src.llm import SYSTEM_PROMPTS, USER_PROMPT_TEMPLATES
SYSTEM_PROMPTS

{'business_summary': '# Identity\n\nYou are a business analyst creating a comprehensive summary of a company based on its\n10-K filing.\n\n## Instructions\n\nYour task is to create a clear, concise overview of the company that includes:\n1. Core business model and revenue streams\n2. Market position and competitive landscape\n3. Key products and services\n4. Growth strategy and future outlook\n5. Major business segments and their performance\n\nFocus on providing a balanced, objective summary that captures the essential aspects of the\nbusiness without subjective evaluation.\n\n## Format Requirements\n\nWrite your output in markdown format:\n- Use hashtags (#, ##, ###, ####, #####) to indicate section headings.\n- Use bold text to emphasize keywords or phrases (**something important**).\n- Use italic text to emphasize phrasing (_must_, _always_, _etc_)\n- Use hyphens (-) to indicate list items.\n- Limit the length of list items to one sentence.\n- Use backticks (`) to highlight equatio

In [34]:
USER_PROMPT_TEMPLATES

{'risk_analysis': "\nPlease analyze the following Risk Factors section from {company_name}'s {year} 10-K filing:\n\n{content}\n",
 'management_assessment': "\nPlease analyze the following Management's Discussion and Analysis section from {company_name}'s {year} 10-K filing:\n\n{content}\n",
 'business_summary': "\nPlease provide a business summary based on the following information from {company_name}'s {year} 10-K filing:\n\n{content}\n"}

In [2]:
from src.database import init_db, get_document
init_db(settings.database.url)

document = get_document(document_id="068074ef-7894-7165-8000-672624774154")

ArgumentError: 'SchemaItem' object, such as a 'Column' or a 'Constraint' expected, got <class 'str'>